### 자소서 생성기


1. 기업 분석 및 직무 분석 (되게 자잘한 것까지)
2. 내 경험 모듈, (1) 내용 바탕으로 경험 모듈 가장 적합할만한 경험 n개 뽑기
3. (2)의 내용과 자소서 문항 및 constraint 고려해서 초안 작성
---> 이때 모든 자소서 문항 한번에 작성되게 만들까 따로따로 만들게 할까
4. 내가 강조하고 싶은 부분을 피드백으로 받고 constraint에 맞게 (3) 내용 재작성

In [5]:
from openai import OpenAI
import os
import dotenv
import glob

In [ ]:
## dotenv load
dotenv.load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=openai_api_key)

In [10]:
### RAG 시키기
# 1) 벡터스토어 생성
vs = client.vector_stores.create(name="how-to-get-a-job")

# 2) 파일 업로드 + 벡터스토어에 넣고(자동 청킹/임베딩) 완료까지 폴링
file_paths = glob.glob("./docs/**", recursive=True)
file_paths = [p for p in file_paths if os.path.isfile(p)]

file_streams = [open(p, "rb") for p in file_paths]
client.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vs.id,
    files=file_streams,
)

VectorStoreFileBatch(id='vs_6965f00430b48191baf0796adc491691', created_at=1768288260, file_counts=FileCounts(cancelled=0, completed=14, failed=0, in_progress=0, total=14), object='vector_store', status='completed', vector_store_id=None, name='how-to-get-a-job', description=None, usage_bytes=0, expires_after=None, expires_at=None, last_active_at=1768288263, metadata={})

In [11]:
model = 'gpt-5-mini-2025-08-07'

response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "키움증권 파생상품 트레이딩 직무에 지원하려고 해. 나의 이력서와 회사의 채용공고를 바탕으로 커버레터를 작성해줘. 최대한 구체적으로 작성해줘."},
    ]
)